In [7]:
print("hello")
%pip install langchain==0.0.251 --quiet --root-user-action=ignore
%pip install faiss-cpu==1.7.4 --quiet --root-user-action=ignore
%pip install pypdf==3.15.1 --quiet --root-user-action=ignore

hello
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Next, we download PDFs locally

In [9]:
from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2022-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2021, source=filenames[0]),
    dict(year=2022, source=filenames[1])
]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

Need to removve end 1997 letter to shareholders from PDFs as it is attached to every one

In [10]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    
    for pagenum in range(len(pdf_reader.pages) -3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)
        
    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()
        

In [11]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]
        
    documents += document
    
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 100,
)

docs = text_splitter.split_documents(documents)

print(f'# of document pages {len(documents)}')
print(f'# of document chunks: {len(docs)}')

# of document pages 13
# of document chunks: 177


Deploy model for embedding

In [12]:
embedding_model_endpoint_name = 'jumpstart-dft-hf-textembedding-all-minilm-l6-v2'


In [13]:
import boto3
aws_region = boto3.Session().region_name

In [18]:
from typing import Dict, List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
import json

class CustomEmbeddingsContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"text_inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")
    
    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]
    
embeddings_content_handler = CustomEmbeddingsContentHandler()

embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=embedding_model_endpoint_name,
    region_name=aws_region,
    content_handler=embeddings_content_handler,
)

In [16]:
from langchain.schema import Document
from langchain.vectorstores import FAISS

In [19]:
db = FAISS.from_documents(docs, embeddings)

In [20]:
query = "Why is Amazon successful?"

In [21]:
results_with_scores = db.similarity_search_with_score(query)

for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

Content: position us well to pursue this large market segment. Amazon Business allows businesses, municipalities,and organizations to procure products like office supplies and other bulk items easily and at great savings.While some areas of the economy have struggled over the past few years, Amazon Business has thrived. Why?Because the team has translated what it means to deliver selection, value, and convenience into a businessprocurement setting, constantly listening to and learning from customers, and innovating
Metadata: {'year': 2022, 'source': 'AMZN-2022-Shareholder-Letter.pdf'}
Score: 0.39322662353515625


Content: When I joined Amazon in 1997, we had booked $15M in revenue in 1996, were a books-only retailer, didnot have a third-party marketplace, and only shipped to addresses in the US. Today, Amazon sells nearly everyphysical and digital retail item you can imagine, with a vibrant third-party seller ecosystem that accountsfor 60% of our unit sales, and reaches customers in vi

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """
<s>[INST] <<SYS>>
Use the context provided to answer the question at the end. If you don't know the answer just say that you don't know, don't try to make up an answer.
<</SYS>>

Context:
--------------
{context}
--------------

Question: {question} [/INST]
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [ ]:
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
import json

class QAContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps(
            {"inputs": [[
                {
                    "role": "system",
                    "content": "",
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ]],
            "parameters": (**model_kwargs)
            })
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return respons_json[0]["generation"]["content"]
    
qa_content_handler = QAContentHandler()

In [ ]:
llm_model_endpoint_name =

In [ ]:
llm = SagemakerEndpoint(
    endpoint_name=llm_model_endpoint_name,
    region_name=aws_region,
    model_kwargs={"max_new_tokens": 1000, "top_p": 0.9, "temperature": 1e-11},
    endpoint_kwargs={"CustomAttributes": 'acept_eula=true'},
    content_handler=qa_content_handler
)

In [ ]:
query = ""
llm.predict(query)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
query = ""
result = qa_chain({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
    print(f'{srcdoc}\n')